# Um caminhar sobre o Python

In [ ]:
import pandas as pd

In [ ]:
# Endereço dos dados a serem trabalhados
url = "https://raw.githubusercontent.com/igormorgado/logprog/main/empregados/empregados.csv"

In [ ]:
# Carrega os dados e exibe para verificação
empregados_df = pd.read_csv(url, index_col=0)
empregados_df

# Conexão com Banco de Dados

In [ ]:
# Carrega a Biblioteca de banco de dados
import sqlite3 as db

In [ ]:
# Conecta-se ao banco de dados em memória
conn = db.connect(':memory:')

# Salva o dataframe pandas no banco de dados
empregados_df.to_sql(name='empregados', con=conn)

## Preparando a requisição

In [ ]:
# Vamos verificar os maiores salários da empresa, preparando a requisição
query = 'SELECT * FROM empregados WHERE salary > 5000 ORDER BY salary DESC'

## Método SQL via Pandas

In [ ]:
pd.read_sql(query, con=conn)

# Gerando Gráficos para Relatórios

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
empregados_df.hist(column=['age', 'salary']);

# Inteligência artificial (Bônus)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pandas_datareader as web
import datetime as dt
import matplotlib.dates as mdates

from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,LSTM

In [ ]:
company = 'FB'

start = dt.datetime(2012,1,1)
end = dt.datetime(2020,1,1)
data = web.DataReader(company, 'stooq', start, end)

In [ ]:
data

In [ ]:
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(data['Close'].values.reshape(-1,1))

prediction_days = 60

x_train = []
y_train = []

for x in range(prediction_days, len(scaled_data)):
  x_train.append(scaled_data[x - prediction_days:x, 0])
  y_train.append(scaled_data[x, 0])

x_train, y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

model = Sequential(name='stock_prediction')
model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50))
model.add(Dropout(0.2))
model.add(Dense(units=1))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(x_train, y_train, epochs=25, batch_size=32)

In [ ]:
test_start = dt.datetime(2020,1,1)
test_end = dt.datetime.now()
test_data = web.DataReader(company, 'stooq', test_start, test_end)
actual_prices = test_data['Close'].values

total_dataset = pd.concat((data['Close'], test_data['Close']), axis=0)

model_inputs = total_dataset[len(total_dataset) - len(test_data) - prediction_days:].values
model_inputs = model_inputs.reshape(-1, 1)
model_inputs = scaler.transform(model_inputs)

## Gerando previsões de teste

In [ ]:
x_test = []

for x in range(prediction_days, len(model_inputs)):
  x_test.append(model_inputs[x - prediction_days:x, 0])

x_test = np.array(x_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

In [ ]:
predicted_prices = model.predict(x_test)

In [ ]:
predicted_prices = scaler.inverse_transform(predicted_prices)

## Gráfico das previsões de teste

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(10,6))

ax.plot(test_data.index, actual_prices, color='k', label=f'Real data')
ax.plot(test_data.index, predicted_prices, color='green', label=f'Predicted data')
ax.set_title(f"{company} share close price")

ax.set_xlabel('Time')
ax.xaxis.set_major_formatter(
    mdates.ConciseDateFormatter(ax.xaxis.get_major_locator()))
ax.xaxis.set_major_locator(mdates.MonthLocator(bymonth=(1, 7)))
ax.xaxis.set_minor_locator(mdates.MonthLocator())
ax.grid(True)
ax.set_ylabel(f'USD')
plt.legend()
plt.show()

## E amanhã?

In [ ]:
real_data = [model_inputs[len(model_inputs) - prediction_days:len(model_inputs + 1), 0]]
real_data = np.array(real_data)
real_data = np.reshape(real_data, (real_data.shape[0], real_data.shape[1], 1))

In [ ]:
real_data.shape

In [ ]:
len(model_inputs)

In [ ]:
prediction = model.predict(real_data)
prediction = scaler.inverse_transform(prediction)
print(f"Prediction {prediction}")